In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [38]:
url_inicial = 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive/4'
url_root = 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive/4'
r=requests.get(url_inicial)

In [39]:
soup = BeautifulSoup(r.text, 'html.parser')

In [40]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/12',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/11']

In [41]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [42]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 

Estoy en la pagina https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive/4


In [43]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

2

In [44]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')


KeyboardInterrupt



In [ ]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Hidrobiológica'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Recursos hidrobiológicos, aspectos fisiológicos y biológicos de los ambientes acuáticos.'
    url='http://investigacion.izt.uam.mx/rehb/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='description').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [45]:
list_scraper=list_scraper[0:2]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1


In [46]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Vol. 1 Núm. 2 (1992),None,http://investigacion.izt.uam.mx/rehb/,https://hidrobiologica.izt.uam.mx/index.php/re...
1,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Vol. 1 Núm. 1 (1991),None,http://investigacion.izt.uam.mx/rehb/,https://hidrobiologica.izt.uam.mx/index.php/re...


In [47]:
df_catalogo.to_csv('Revista41.04.csv', index=False)